In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from typing import Union, List, Callable, Optional, Dict, Any
import json
from utils import get_colors

# Sorbetto: a Python Library to Produce Classification Tiles With Different Flavors

Code in [![GitHub stars](https://img.shields.io/github/stars/pierard/sorbetto?style=social)](https://github.com/pierard/sorbetto)

## Theoretical Analyst 

The theoretical analyst is mainly interested in the foundations of evaluation. Instead of focusing on a particular model or application, they want to understand how different performance scores relate to one another. Their goal is to study the theoretical relationships between evaluation metrics, see how they correlate, and identify whether scores provide complementary or redundant information. In short, they care about the theory behind performance measures.

In this example, it is shown how the tile can be used for such objective.


For more information, please refer to the  [*reference paper*](https://arxiv.org/abs/2412.04377).

### Installing the library

If you are using google colab, it is possible to install the library by running the code below. If it is not the case, please refer to install.md to get a local installation.

In [3]:
# only for google colab

CODE_DIR = 'sorbetto'
os.makedirs(f'./{CODE_DIR}')
!git clone git@github.com:pierard/sorbetto.git $CODE_DIR
os.chdir(f'./{CODE_DIR}')

!bash install.sh

FileExistsError: [Errno 17] File exists: './sorbetto'

### Data

In this example, the list of entities are the different deep learning models from mmsegmentation that are obtained on cityscape data.

In [2]:
# loading data

from sorbetto.core.entity import Entity
from sorbetto.performance.two_class_classification import TwoClassClassificationPerformance

PATH_DATA = os.path.join(os.getcwd(), 'data', 'semantic_segmentation_methods.json')

MAP_DATASETS = {
    'Cityscapes': '♠',
    'ADE20K': '♥',
    'Pascal VOC 2012': '♦',
    'COCO-Stuff 164k': '♣'
}

def read_json(filePath:str):
    """
    Function to read json 
    """
    
    with open(filePath, 'r') as f:
        data = json.load(f)
    return data


def read_data(filePath:str = PATH_DATA):
    data = read_json(filePath)

    processed_data = []

    colors = get_colors(len(data))

    # convert each instance into an Entity
    for i, elem in enumerate(data):
        performance = TwoClassClassificationPerformance(
            ptp=elem['proba_tp'],
            ptn=elem['proba_tn'],
            pfp=elem['proba_fp'],
            pfn=elem['proba_fn']
        )

        entity = Entity(
            performance= performance,
            name = f"{elem['model']} {MAP_DATASETS[elem['learning_set']]}",
            color= colors[i]
        )

        processed_data.append(entity)

    return processed_data



data = read_data()

In [3]:
print(data[0])

Entity `ANN ♠` with performance 
 TwoClassClassificationPerformance(name=Two class P, ptn=0.8390197560439461, pfp=0.03675330043250826, pfn=0.01777933853901557, ptp=0.1064476049845301)


In [ ]:
from sorbetto.ranking.ranking_score import RankingScore
from sorbetto.parametrization.parametrization_default import ParameterizationDefault

parameterization = ParameterizationDefault()
score = RankingScore.getAccuracy()

performance_set = FiniteSetOfTwoClassClassificationPerformances(
    [entity.evaluate() for entity in data]
    )

resolution = 1001

In [ ]:
from sorbetto.analyses.analysis_theoretical_analyst import AnalysisForTheoreticalAnalyst

theoretical_analyst = AnalysisForTheoreticalAnalyst(
    performances=performance_set,
    parameterization=parameterization,
    score=score,
    resolution=resolution
)


corr_tile = theoretical_analyst.getKendallCorrelationTile()

In [ ]:
correlations = ["getPearsonCorrelationTile",
               "getKendallCorrelationTile",
               "getSpearmanCorrelationTile"]

res = []

for corr in correlations:
    r = eval(f"theoretical_analyst.{corr}()")
    res.append(r)